In [1]:
import numpy as np
import pandas as pd

movie_data = pd.read_csv("../movie_rating/IMDb Movies India.csv")

In [2]:
mask = movie_data.isnull()
for i in range(mask.shape[0]):
    for bool in mask.iloc[i].values:
        if bool == True:
            movie_data.drop(i, inplace=True)
            break
movie_data.reset_index(inplace=True, drop=True)

In [3]:
for i in range (movie_data["Genre"].shape[0]):
    movie_data["Genre"][i] = movie_data["Genre"][i].split(",")

for  i in range (movie_data["Genre"].shape[0]):
    for j , word in enumerate(movie_data["Genre"][i]):
        movie_data["Genre"][i][j] = word.replace(" ", '')

/tmp/ipykernel_2685/2108306218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data["Genre"][i] = movie_data["Genre"][i].split(",")
/tmp/ipykernel_2685/2108306218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data["Genre"][i] = movie_data["Genre"][i].split(",")
/tmp/ipykernel_2685/2108306218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data["Genre"][i] = movie_data["Genre"][i].split(",")
/tmp/ipykerne

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
genre_encoding =MultiLabelBinarizer()
transform = genre_encoding.fit_transform(movie_data["Genre"])

In [5]:
for i , gerne in enumerate(genre_encoding.classes_):
    movie_data[gerne] = transform[:,i]

In [6]:
movie_data.drop("Genre", axis=1, inplace=True)

In [7]:
movie_data["Duration"] = movie_data["Duration"].replace("min", '', regex=True).astype("int")
movie_data["Year"] = movie_data["Year"].replace(r'\(|\)', '', regex=True).astype("int")
movie_data["Votes"] = movie_data["Votes"].replace(",", '', regex=True).astype("int")

In [8]:
list(movie_data["Votes"] < 500000).count(True)

5656

In [9]:
def remove_outliers(df,columns,n_std):
    for col in columns:
        print('Working on column: {}'.format(col))
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

In [10]:
movie_data = remove_outliers(movie_data, ["Year", "Duration","Votes", "Rating"], 2)


Working on column: Year
Working on column: Duration
Working on column: Votes
Working on column: Rating


In [11]:
""" import matplotlib.pyplot as plt

plt.plot(movie_data["Year"], movie_data["Rating"], 'o') """

' import matplotlib.pyplot as plt\n\nplt.plot(movie_data["Year"], movie_data["Rating"], \'o\') '

In [12]:
y = movie_data.pop("Rating")

In [13]:
catigorical_featutes = ["Name", "Director", "Actor 1", "Actor 2", "Actor 3"]

numerical_features = ["Year", "Duration","Votes"]
numerical_features.extend(list(genre_encoding.classes_))

from sklearn import preprocessing


features = []
for category in catigorical_featutes:
    vocab = movie_data[category].unique()
    vocab = vocab.reshape(len(vocab), 1)
    enc = preprocessing.OrdinalEncoder()
    enc.fit(vocab)
    features.append(enc.transform(np.array(movie_data[category]).reshape(len(movie_data[category]), 1)))

for numeric in numerical_features:
    features.append(np.array(movie_data[numeric]).reshape(len(movie_data[numeric]), 1))
    

In [14]:
features_colums = np.concatenate(features, axis=1) # input feature

In [15]:
# splite the data into train and test sets 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_colums, y, test_size=0.33, random_state=42)

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

reg = LinearRegression().fit(X_train, y_train)

In [17]:
pred = reg.predict(X_test)
score = r2_score(y_test, pred)
score

0.24122757884010426

In [18]:


plt.plot(X_train[:,2], y_train, 'o')
""" plt.plot(X_train, reg.coef_ * X_train + reg.intercept_) """
plt.show()

NameError: name 'plt' is not defined